## Import Package

In [66]:
import numpy as np
import pandas as pd
import os

# import seaborn as sns
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset

## Load Data

In [67]:
os.getcwd()

'/home/sagemaker-user/capstone-2024-summer/src/shuo'

In [68]:
!pwd

/home/sagemaker-user/capstone-2024-summer/src/shuo


In [69]:
df = pd.read_parquet("../../data/crsp_2018-2023_clean_3.parquet")

In [70]:
df

,date,permno_id,ncusip_id,cusip_id,ticker,share_code,share_code_type,share_code_detail,exchange_code,company_name,...,value_weighted_return,equal_weighted_return,return_on_SP_index,log_return,market_cap,weighted_price,sector_weighted_avg_log_return,sector_simple_avg_log_return,weighted_avg_log_return,simple_avg_log_return
9263,2018-01-02,10104,68389X10,68389X10,ORCL,11.0,1,1,1,ORACLE CORP,...,0.008505,0.011232,0.008303,-0.013843,1.930296e+08,9.000972e+09,0.000000,0.000000,-0.0,-0.00064
9264,2018-01-03,10104,68389X10,68389X10,ORCL,11.0,1,1,1,ORACLE CORP,...,0.005856,0.005103,0.006399,0.022897,1.975004e+08,9.422745e+09,0.021973,0.005694,0.0,-0.00064
9265,2018-01-04,10104,68389X10,68389X10,ORCL,11.0,1,1,1,ORACLE CORP,...,0.004001,0.005272,0.004029,0.009803,1.994460e+08,9.609309e+09,0.002940,0.001681,0.0,-0.00064
9266,2018-01-05,10104,68389X10,68389X10,ORCL,11.0,1,1,1,ORACLE CORP,...,0.005804,0.003880,0.007034,0.006001,2.006465e+08,9.725336e+09,0.015718,0.004005,0.0,-0.00064
9267,2018-01-08,10104,68389X10,68389X10,ORCL,11.0,1,1,1,ORACLE CORP,...,0.001816,0.000584,0.001662,0.010467,2.027577e+08,9.931072e+09,0.006267,0.009789,0.0,-0.00064
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7658917,2023-12-22,93436,88160R10,88160R10,TSLA,11.0,1,1,3,TESLA INC,...,0.002440,0.007676,0.001660,-0.007731,8.028047e+08,2.027403e+11,-0.001753,0.016253,-0.0,-0.00064
7658918,2023-12-26,93436,88160R10,88160R10,TSLA,11.0,1,1,3,TESLA INC,...,0.005218,0.009674,0.004232,0.015988,8.157429e+08,2.093278e+11,0.006185,0.001010,0.0,-0.00064
7658919,2023-12-27,93436,88160R10,88160R10,TSLA,11.0,1,1,3,TESLA INC,...,0.001995,0.005276,0.001430,0.018647,8.310971e+08,2.172820e+11,0.003881,0.001242,0.0,-0.00064
7658920,2023-12-28,93436,88160R10,88160R10,TSLA,11.0,1,1,3,TESLA INC,...,-0.000108,0.002042,0.000370,-0.032104,8.048392e+08,2.037692e+11,-0.002012,0.004141,-0.0,-0.00064


In [71]:
df.dtypes

date                              datetime64[ns]
permno_id                                 object
ncusip_id                                 object
cusip_id                                  object
ticker                                    object
share_code                                object
share_code_type                           object
share_code_detail                         object
exchange_code                             object
company_name                              object
primary_exchange                          object
trading_status                            object
security_status                           object
naics                                     object
naics_sector                              object
naics_secondary                           object
ask_or_high_price                        float64
ask_price                                float64
bid_or_low_price                         float64
bid_price                                float64
open_price          

In [72]:
max(df.groupby("permno_id")["ticker"].nunique())

3

In [73]:
# Group by 'PERMNO' and count unique 'TICKER', then filter for those with exactly 2 unique 'TICKER'
permno_with_multiple_tickers = df.groupby("permno_id")["ticker"].nunique()
permno_with_multiple_tickers = permno_with_multiple_tickers[
    permno_with_multiple_tickers == 2
].index

# Display the PERMNO values
# data[data['PERMNO'].isin(permno_with_multiple_tickers)]
permno_with_multiple_tickers

Index(['10696', '12431', '13407', '14882', '16048', '16692', '16851', '17830',
       '19751', '24053', '24643', '25582', '42200', '52329', '57568', '62308',
       '64282', '67598', '71563', '75034', '76082', '81540', '82303', '86783',
       '89017', '89179', '90199', '90441', '90442', '91277'],
      dtype='object', name='permno_id')

In [74]:
df[df["permno_id"].isin(permno_with_multiple_tickers)]

,date,permno_id,ncusip_id,cusip_id,ticker,share_code,share_code_type,share_code_detail,exchange_code,company_name,...,value_weighted_return,equal_weighted_return,return_on_SP_index,log_return,market_cap,weighted_price,sector_weighted_avg_log_return,sector_simple_avg_log_return,weighted_avg_log_return,simple_avg_log_return
68278,2018-01-02,10696,33773810,33773810,FISV,11.0,1,1,3,FISERV INC,...,0.008505,0.011232,0.008303,-0.005966,2.706066e+07,3.527358e+09,0.000000,0.000000,-0.0,-0.00064
68279,2018-01-03,10696,33773810,33773810,FISV,11.0,1,1,3,FISERV INC,...,0.005856,0.005103,0.006399,0.008099,2.728072e+07,3.584959e+09,0.021973,0.005694,0.0,-0.00064
68280,2018-01-04,10696,33773810,33773810,FISV,11.0,1,1,3,FISERV INC,...,0.004001,0.005272,0.004029,0.013980,2.766477e+07,3.686607e+09,0.002940,0.001681,0.0,-0.00064
68281,2018-01-05,10696,33773810,33773810,FISV,11.0,1,1,3,FISERV INC,...,0.005804,0.003880,0.007034,0.009857,2.793881e+07,3.760005e+09,0.015718,0.004005,0.0,-0.00064
68282,2018-01-08,10696,33773810,33773810,FISV,11.0,1,1,3,FISERV INC,...,0.001816,0.000584,0.001662,0.002004,2.799486e+07,3.775107e+09,0.006267,0.009789,0.0,-0.00064
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7142596,2023-12-22,91277,74915M10,74915M10,QRTEA,11.0,1,1,3,QURATE RETAIL INC,...,0.002440,0.007676,0.001660,-0.017352,3.305973e+05,2.870907e+05,0.003961,0.005242,-0.0,-0.00064
7142597,2023-12-26,91277,74915M10,74915M10,QRTEA,11.0,1,1,3,QURATE RETAIL INC,...,0.005218,0.009674,0.004232,0.000346,3.307115e+05,2.872891e+05,0.003031,0.004622,0.0,-0.00064
7142598,2023-12-27,91277,74915M10,74915M10,QRTEA,11.0,1,1,3,QURATE RETAIL INC,...,0.001995,0.005276,0.001430,-0.001037,3.303689e+05,2.866941e+05,-0.002365,0.007425,-0.0,-0.00064
7142599,2023-12-28,91277,74915M10,74915M10,QRTEA,11.0,1,1,3,QURATE RETAIL INC,...,-0.000108,0.002042,0.000370,0.030863,3.407238e+05,3.049478e+05,-0.000601,0.003489,0.0,-0.00064


## Feature Engineering

In [80]:
data = df[["date", "permno_id", "log_return"]]
data

,date,permno_id,log_return
9263,2018-01-02,10104,-0.013843
9264,2018-01-03,10104,0.022897
9265,2018-01-04,10104,0.009803
9266,2018-01-05,10104,0.006001
9267,2018-01-08,10104,0.010467
...,...,...,...
7658917,2023-12-22,93436,-0.007731
7658918,2023-12-26,93436,0.015988
7658919,2023-12-27,93436,0.018647
7658920,2023-12-28,93436,-0.032104


## Model

In [81]:
data.groupby("permno_id")["date"].describe()

,count,mean,min,25%,50%,75%,max
permno_id,,,,,,,
10104,1509,2020-12-30 14:13:07.276342016,2018-01-02 00:00:00,2019-07-03 00:00:00,2020-12-30 00:00:00,2022-06-30 00:00:00,2023-12-29 00:00:00
10107,1509,2020-12-30 14:13:07.276342016,2018-01-02 00:00:00,2019-07-03 00:00:00,2020-12-30 00:00:00,2022-06-30 00:00:00,2023-12-29 00:00:00
10138,1509,2020-12-30 14:13:07.276342016,2018-01-02 00:00:00,2019-07-03 00:00:00,2020-12-30 00:00:00,2022-06-30 00:00:00,2023-12-29 00:00:00
10145,1509,2020-12-30 14:13:07.276342016,2018-01-02 00:00:00,2019-07-03 00:00:00,2020-12-30 00:00:00,2022-06-30 00:00:00,2023-12-29 00:00:00
10516,1509,2020-12-30 14:13:07.276342016,2018-01-02 00:00:00,2019-07-03 00:00:00,2020-12-30 00:00:00,2022-06-30 00:00:00,2023-12-29 00:00:00
...,...,...,...,...,...,...,...
93096,1509,2020-12-30 14:13:07.276342016,2018-01-02 00:00:00,2019-07-03 00:00:00,2020-12-30 00:00:00,2022-06-30 00:00:00,2023-12-29 00:00:00
93132,1509,2020-12-30 14:13:07.276342016,2018-01-02 00:00:00,2019-07-03 00:00:00,2020-12-30 00:00:00,2022-06-30 00:00:00,2023-12-29 00:00:00
93246,1509,2020-12-30 14:13:07.276342016,2018-01-02 00:00:00,2019-07-03 00:00:00,2020-12-30 00:00:00,2022-06-30 00:00:00,2023-12-29 00:00:00


In [83]:
first_month_data = data[data["date"] < "2018-02-01"]
first_month_data

,date,permno_id,log_return
9263,2018-01-02,10104,-0.013843
9264,2018-01-03,10104,0.022897
9265,2018-01-04,10104,0.009803
9266,2018-01-05,10104,0.006001
9267,2018-01-08,10104,0.010467
...,...,...,...
7657429,2018-01-25,93436,-0.024141
7657430,2018-01-26,93436,0.015313
7657431,2018-01-29,93436,0.019297
7657432,2018-01-30,93436,-0.010671


### Check and make sure no missing values

In [84]:
first_month_data.dtypes

date          datetime64[ns]
permno_id             object
log_return           float64
dtype: object

In [85]:
first_month_data[first_month_data["log_return"].isnull() == 1]

,date,permno_id,log_return


In [86]:
first_month_data[first_month_data["date"].isnull() == 1]

,date,permno_id,log_return


In [87]:
first_month_data.isnull().sum()

date          0
permno_id     0
log_return    0
dtype: int64

In [88]:
len(first_month_data)

11025

### Build Model & Get Embedding

#### Model-1
1. The current model does not use positional encodings.
2. Need to add ordinal position for each return.
3. Need to have 1 modified transformer encoder layer without residual connections + (n-1) standard layers
4. Need to connect the output of the first transformer block to the embedding layer of the fi- nal encoder block

In [90]:
first_month_data[:100]

,date,permno_id,log_return
9263,2018-01-02,10104,-0.013843
9264,2018-01-03,10104,0.022897
9265,2018-01-04,10104,0.009803
9266,2018-01-05,10104,0.006001
9267,2018-01-08,10104,0.010467
...,...,...,...
51606,2018-01-18,10516,-0.010567
51607,2018-01-19,10516,0.011788
51608,2018-01-22,10516,0.042773
51609,2018-01-23,10516,-0.003280


In [95]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np


class FinancialDataset(Dataset):
    def __init__(self, data, window_size=7):
        self.data = data
        self.window_size = window_size
        self.data["log_return"] = pd.to_numeric(
            self.data["log_return"], errors="coerce"
        )
        self.data = self.data.dropna()

        # Group by permno_id
        self.groups = self.data.groupby("permno_id")
        self.sequences = []

        for name, group in self.groups:
            if len(group) > self.window_size:
                for i in range(len(group) - self.window_size):
                    X = group["log_return"].iloc[i : i + self.window_size].values
                    y = group["log_return"].iloc[i + self.window_size]
                    self.sequences.append((X, y, name))

    def __len__(self):
        return len(self.sequences)

    def __getitem__(self, idx):
        X, y, permno_id = self.sequences[idx]
        return {
            "features": torch.tensor(X, dtype=torch.float),
            "target": torch.tensor(y, dtype=torch.float),
            "permno_id": permno_id,
        }


class TransformerModel(nn.Module):
    def __init__(self, input_dim, embed_dim, n_heads, ff_dim, n_layers):
        super(TransformerModel, self).__init__()
        self.embedding = nn.Linear(input_dim, embed_dim)
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=embed_dim, nhead=n_heads, dim_feedforward=ff_dim
        )
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=n_layers)
        self.fc = nn.Linear(embed_dim, 1)

    def forward(self, x):
        x = self.embedding(x)  # Apply embedding layer
        x = x.unsqueeze(
            1
        )  # Add a dimension for the transformer (batch_size, seq_len, embed_dim)
        x = self.transformer(
            x
        )  # Transformer expects input of shape (seq_len, batch_size, embed_dim)
        # return self.fc(x[-1])
        embedding = x[:, -1, :]  # Get the last output token
        x = self.fc(x[:, -1, :])  # Fully connected layer on the last output token
        return x.squeeze(-1), embedding  # Return the final output and the embeddings


# Prepare Data
dataset = FinancialDataset(first_month_data[:100])
dataloader = DataLoader(dataset, batch_size=64, shuffle=False)

# Initialize Model
model = TransformerModel(input_dim=7, embed_dim=64, n_heads=8, ff_dim=256, n_layers=4)

# Training Loop
optimizer = torch.optim.RAdam(model.parameters(), lr=0.001)
# criterion = nn.MSELoss()
criterion = nn.L1Loss()

for epoch in range(1):
    for batch in dataloader:
        optimizer.zero_grad()
        output = model(batch["features"])
        loss = criterion(output[0].squeeze(), batch["target"])
        loss.backward()
        optimizer.step()

print("Training Complete")

Training Complete


/tmp/ipykernel_155/3712811391.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.data['log_return'] = pd.to_numeric(self.data['log_return'], errors='coerce')
/opt/conda/lib/python3.10/site-packages/torch/nn/modules/loss.py:101: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)


In [96]:
# Extract embeddings for each stock
stock_embeddings = {}
model.eval()
with torch.no_grad():
    for batch in dataloader:
        _, embeddings = model(batch["features"])
        for i, permno_id in enumerate(batch["permno_id"]):
            if permno_id not in stock_embeddings:
                stock_embeddings[permno_id] = []
            stock_embeddings[permno_id].append(embeddings[i].cpu().numpy())

In [97]:
# Average the embeddings for each stock
for permno_id in stock_embeddings:
    stock_embeddings[permno_id] = np.mean(stock_embeddings[permno_id], axis=0)

In [98]:
stock_embeddings

{'10104': array([ 1.9125372 ,  1.7963878 ,  0.8690127 ,  1.0095726 ,  0.8097637 ,
         1.4758785 ,  0.3846818 ,  0.5171198 , -0.5502455 , -0.79582125,
         0.9149669 , -0.72370964,  0.2157372 , -0.40707904, -0.3798963 ,
         0.0681246 , -0.18942799,  0.30252597, -1.6335363 , -0.50203407,
         1.067426  , -0.01582441,  1.4036318 ,  0.03825132, -0.7245344 ,
        -1.2946919 ,  1.0396432 ,  1.4527997 ,  0.24460371, -0.4906159 ,
         0.90415394,  1.6392214 ,  1.8191874 , -0.6335238 , -1.2813393 ,
         0.4137523 , -1.305227  , -0.31172892, -0.5520805 ,  1.4794992 ,
         0.23203723,  0.697966  , -0.48349994,  1.0807937 ,  0.03367883,
         0.52996093, -2.9019568 , -1.3881916 , -0.85196   , -1.2370913 ,
         0.62988555, -1.1565157 , -0.10638807, -0.39140677, -0.15645273,
        -0.03827002, -1.5450052 , -1.5472907 , -0.601257  ,  0.19124922,
        -0.9981534 , -0.7941331 , -0.01237456,  0.8274766 ], dtype=float32),
 '10107': array([ 1.9099072 ,  1.80026

In [99]:
for batch in dataloader:
    print(batch)
    break

{'features': tensor([[-0.0138,  0.0229,  0.0098,  0.0060,  0.0105,  0.0055, -0.0053],
        [ 0.0229,  0.0098,  0.0060,  0.0105,  0.0055, -0.0053,  0.0031],
        [ 0.0098,  0.0060,  0.0105,  0.0055, -0.0053,  0.0031,  0.0114],
        [ 0.0060,  0.0105,  0.0055, -0.0053,  0.0031,  0.0114,  0.0016],
        [ 0.0105,  0.0055, -0.0053,  0.0031,  0.0114,  0.0016,  0.0136],
        [ 0.0055, -0.0053,  0.0031,  0.0114,  0.0016,  0.0136, -0.0008],
        [-0.0053,  0.0031,  0.0114,  0.0016,  0.0136, -0.0008,  0.0069],
        [ 0.0031,  0.0114,  0.0016,  0.0136, -0.0008,  0.0069,  0.0026],
        [ 0.0114,  0.0016,  0.0136, -0.0008,  0.0069,  0.0026,  0.0081],
        [ 0.0016,  0.0136, -0.0008,  0.0069,  0.0026,  0.0081,  0.0064],
        [ 0.0136, -0.0008,  0.0069,  0.0026,  0.0081,  0.0064,  0.0029],
        [-0.0008,  0.0069,  0.0026,  0.0081,  0.0064,  0.0029,  0.0220],
        [ 0.0069,  0.0026,  0.0081,  0.0064,  0.0029,  0.0220, -0.0155],
        [ 0.0026,  0.0081,  0.0064,  0

In [100]:
model(batch["features"])

(tensor([0.0594, 0.0633, 0.0687, 0.0598, 0.0639, 0.0705, 0.0744, 0.0619, 0.0713,
         0.0594, 0.0687, 0.0696, 0.0616, 0.0672, 0.0692, 0.0686, 0.0613, 0.0584,
         0.0757, 0.0701, 0.0729, 0.0573, 0.0782, 0.0577, 0.0684, 0.0786, 0.0537,
         0.0623, 0.0662, 0.0716, 0.0682, 0.0495, 0.0606, 0.0623, 0.0733, 0.0502,
         0.0760, 0.0572, 0.0739, 0.0660, 0.0583, 0.0618, 0.0792, 0.0631, 0.0609,
         0.0722, 0.0705, 0.0639, 0.0680, 0.0591, 0.0748, 0.0662, 0.0748, 0.0665,
         0.0631, 0.0705, 0.0772, 0.0611, 0.0653, 0.0744, 0.0650, 0.0768, 0.0646,
         0.0732], grad_fn=<SqueezeBackward1>),
 tensor([[ 1.9210,  1.7609,  0.8728,  ..., -0.7800, -0.0296,  0.8177],
         [ 1.8955,  1.7882,  0.8892,  ..., -0.7763,  0.0053,  0.8354],
         [ 1.9005,  1.8053,  0.8653,  ..., -0.8027, -0.0198,  0.8212],
         ...,
         [ 1.9233,  1.7903,  0.8683,  ..., -0.8039,  0.0025,  0.8350],
         [ 1.8826,  1.7931,  0.8714,  ..., -0.7969, -0.0193,  0.8236],
         [ 1.9042

In [101]:
dataset.sequences

[(array([-0.01384338,  0.02289685,  0.0098028 ,  0.00600096,  0.01046703,
          0.00549686, -0.00531409]),
  0.0030692849222946,
  '10104'),
 (array([ 0.02289685,  0.0098028 ,  0.00600096,  0.01046703,  0.00549686,
         -0.00531409,  0.00306928]),
  0.0113750580215052,
  '10104'),
 (array([ 0.0098028 ,  0.00600096,  0.01046703,  0.00549686, -0.00531409,
          0.00306928,  0.01137506]),
  0.0016146956770015,
  '10104'),
 (array([ 0.00600096,  0.01046703,  0.00549686, -0.00531409,  0.00306928,
          0.01137506,  0.0016147 ]),
  0.0136188411579912,
  '10104'),
 (array([ 0.01046703,  0.00549686, -0.00531409,  0.00306928,  0.01137506,
          0.0016147 ,  0.01361884]),
  -0.0007963169762198,
  '10104'),
 (array([ 0.00549686, -0.00531409,  0.00306928,  0.01137506,  0.0016147 ,
          0.01361884, -0.00079632]),
  0.0069438356744076,
  '10104'),
 (array([-0.00531409,  0.00306928,  0.01137506,  0.0016147 ,  0.01361884,
         -0.00079632,  0.00694384]),
  0.00256670319731

#### Model-2
add ordinal position for each return

In [102]:
data

,date,permno_id,log_return
9263,2018-01-02,10104,-0.013843
9264,2018-01-03,10104,0.022897
9265,2018-01-04,10104,0.009803
9266,2018-01-05,10104,0.006001
9267,2018-01-08,10104,0.010467
...,...,...,...
7658917,2023-12-22,93436,-0.007731
7658918,2023-12-26,93436,0.015988
7658919,2023-12-27,93436,0.018647
7658920,2023-12-28,93436,-0.032104


In [103]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np


class FinancialDataset(Dataset):
    def __init__(self, data, window_size=7):
        self.data = data
        self.window_size = window_size
        self.data["log_return"] = pd.to_numeric(
            self.data["log_return"], errors="coerce"
        )
        self.data = self.data.dropna()

        # Group by permno_id
        self.groups = self.data.groupby("permno_id")
        self.sequences = []

        for name, group in self.groups:
            j = self.window_size
            if len(group) > self.window_size:
                for i in range(len(group) - self.window_size):
                    X = group["log_return"].iloc[i : i + self.window_size].values
                    X = np.append(X, j)
                    j += 1

                    # print(X)
                    # t = np.arange(1, self.window_size + 1)  # Relative positions
                    # X = np.column_stack((X, t))  # Concatenate returns with positions

                    y = group["log_return"].iloc[i + self.window_size]
                    self.sequences.append((X, y, name))

    def __len__(self):
        return len(self.sequences)

    def __getitem__(self, idx):
        X, y, permno_id = self.sequences[idx]
        return {
            "features": torch.tensor(X, dtype=torch.float),
            "target": torch.tensor(y, dtype=torch.float),
            "permno_id": permno_id,
        }


class TransformerModel(nn.Module):
    def __init__(self, input_dim, embed_dim, n_heads, ff_dim, n_layers):
        super(TransformerModel, self).__init__()
        self.embedding = nn.Linear(input_dim, embed_dim)
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=embed_dim, nhead=n_heads, dim_feedforward=ff_dim
        )
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=n_layers)
        self.fc = nn.Linear(embed_dim, 1)

    def forward(self, x):
        x = self.embedding(x)  # Apply embedding layer
        x = x.unsqueeze(
            1
        )  # Add a dimension for the transformer (batch_size, seq_len, embed_dim)
        x = self.transformer(
            x
        )  # Transformer expects input of shape (seq_len, batch_size, embed_dim)
        embedding = x[:, -1, :]  # Get the last output token
        x = self.fc(embedding)  # Fully connected layer on the last output token
        return x.squeeze(-1), embedding  # Return the final output and the embeddings


# Prepare Data
dataset = FinancialDataset(first_month_data[:100])
dataloader = DataLoader(dataset, batch_size=64, shuffle=False)

# Initialize Model
model = TransformerModel(input_dim=8, embed_dim=64, n_heads=8, ff_dim=256, n_layers=4)


# Training Loop
optimizer = torch.optim.RAdam(model.parameters(), lr=0.001)
criterion = nn.L1Loss()

for epoch in range(1):
    for batch in dataloader:
        optimizer.zero_grad()
        output, _ = model(batch["features"])
        loss = criterion(output, batch["target"])
        # print(f'loss: {loss}')
        loss.backward()
        optimizer.step()

print("Training Complete")

Training Complete


/tmp/ipykernel_155/3163159875.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.data['log_return'] = pd.to_numeric(self.data['log_return'], errors='coerce')


In [104]:
batch["features"]

tensor([[-1.4801e-03,  6.8902e-03,  3.1829e-03, -1.0567e-02,  1.1788e-02,
          4.2773e-02, -3.2804e-03,  1.5000e+01]])

In [105]:
for batch in dataloader:
    print(batch)
    break

{'features': tensor([[-1.3843e-02,  2.2897e-02,  9.8028e-03,  6.0010e-03,  1.0467e-02,
          5.4969e-03, -5.3141e-03,  7.0000e+00],
        [ 2.2897e-02,  9.8028e-03,  6.0010e-03,  1.0467e-02,  5.4969e-03,
         -5.3141e-03,  3.0693e-03,  8.0000e+00],
        [ 9.8028e-03,  6.0010e-03,  1.0467e-02,  5.4969e-03, -5.3141e-03,
          3.0693e-03,  1.1375e-02,  9.0000e+00],
        [ 6.0010e-03,  1.0467e-02,  5.4969e-03, -5.3141e-03,  3.0693e-03,
          1.1375e-02,  1.6147e-03,  1.0000e+01],
        [ 1.0467e-02,  5.4969e-03, -5.3141e-03,  3.0693e-03,  1.1375e-02,
          1.6147e-03,  1.3619e-02,  1.1000e+01],
        [ 5.4969e-03, -5.3141e-03,  3.0693e-03,  1.1375e-02,  1.6147e-03,
          1.3619e-02, -7.9632e-04,  1.2000e+01],
        [-5.3141e-03,  3.0693e-03,  1.1375e-02,  1.6147e-03,  1.3619e-02,
         -7.9632e-04,  6.9438e-03,  1.3000e+01],
        [ 3.0693e-03,  1.1375e-02,  1.6147e-03,  1.3619e-02, -7.9632e-04,
          6.9438e-03,  2.5667e-03,  1.4000e+01],
   

In [106]:
# Extract embeddings for each stock
stock_embeddings = {}
model.eval()
with torch.no_grad():
    for batch in dataloader:
        _, embeddings = model(batch["features"])
        for i, permno_id in enumerate(batch["permno_id"]):
            if permno_id not in stock_embeddings:
                stock_embeddings[permno_id] = []
            stock_embeddings[permno_id].append(embeddings[i].cpu().numpy())

In [108]:
np.array(stock_embeddings["10104"]).shape

(14, 64)

In [109]:
first_month_data[first_month_data["permno_id"] == "10104"]

,date,permno_id,log_return
9263,2018-01-02,10104,-0.013843
9264,2018-01-03,10104,0.022897
9265,2018-01-04,10104,0.009803
9266,2018-01-05,10104,0.006001
9267,2018-01-08,10104,0.010467
9268,2018-01-09,10104,0.005497
9269,2018-01-10,10104,-0.005314
9270,2018-01-11,10104,0.003069
9271,2018-01-12,10104,0.011375
9272,2018-01-16,10104,0.001615


In [110]:
# Average the embeddings for each stock
for permno_id in stock_embeddings:
    stock_embeddings[permno_id] = np.mean(stock_embeddings[permno_id], axis=0)

In [111]:
stock_embeddings

{'10104': array([ 0.8361108 , -1.0824559 ,  1.1948885 , -0.5833731 , -0.7558198 ,
         1.9145528 , -0.99083143, -1.0368159 ,  1.5079815 , -1.5273713 ,
        -0.04817335, -1.8466128 , -0.538207  ,  0.9810921 , -0.02120137,
         0.42843366,  0.4991719 , -0.9739636 , -1.351874  ,  0.7159575 ,
        -0.18188281,  0.03814878, -0.6339988 , -0.44567028, -1.0360154 ,
         0.06057684, -0.08851874, -0.9332918 ,  1.3569211 , -0.8298152 ,
         0.6574249 , -0.60672253, -0.3690898 ,  2.3545687 ,  0.9705063 ,
         0.47866705,  1.788605  , -0.70557076,  0.22626948,  0.17990315,
         1.438578  ,  0.09287889,  1.0884174 , -0.32839423, -1.6315691 ,
         0.80973953,  1.3536078 , -1.3980173 , -0.36037517,  0.6423341 ,
        -0.61527056, -1.8810586 , -1.8648793 ,  1.4509478 ,  0.5705926 ,
         0.7937562 ,  0.37825313, -0.09564893,  0.6315783 , -0.789246  ,
         0.17068441,  0.9858394 , -0.6090761 , -0.435771  ], dtype=float32),
 '10107': array([ 0.836268  , -1.08254

#### Model-3  
customized transformer model

In [112]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np


class FinancialDataset(Dataset):
    def __init__(self, data, window_size=7):
        self.data = data
        self.window_size = window_size
        self.data["log_return"] = pd.to_numeric(
            self.data["log_return"], errors="coerce"
        )
        self.data = self.data.dropna()

        # Group by permno_id
        self.groups = self.data.groupby("permno_id")
        self.sequences = []

        for name, group in self.groups:
            j = self.window_size
            if len(group) > self.window_size:
                for i in range(len(group) - self.window_size):
                    X = group["log_return"].iloc[i : i + self.window_size].values
                    X = np.append(X, j)
                    j += 1

                    # print(X)
                    # t = np.arange(1, self.window_size + 1)  # Relative positions
                    # X = np.column_stack((X, t))  # Concatenate returns with positions

                    y = group["log_return"].iloc[i + self.window_size]
                    self.sequences.append((X, y, name))

    def __len__(self):
        return len(self.sequences)

    def __getitem__(self, idx):
        X, y, permno_id = self.sequences[idx]
        return {
            "features": torch.tensor(X, dtype=torch.float),
            "target": torch.tensor(y, dtype=torch.float),
            "permno_id": permno_id,
        }


class TransformerModel(nn.Module):
    def __init__(self, input_dim, embed_dim, n_heads, ff_dim, n_layers):
        super(TransformerModel, self).__init__()
        self.embedding = nn.Linear(input_dim, embed_dim)
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=embed_dim, nhead=n_heads, dim_feedforward=ff_dim
        )
        self.transformer1 = nn.TransformerEncoder(encoder_layer, num_layers=1)
        self.transformer2 = nn.TransformerEncoder(
            encoder_layer, num_layers=n_layers - 1
        )
        self.fc = nn.Linear(embed_dim, 1)

    def forward(self, x):
        x = self.embedding(x)  # Apply embedding layer
        x = x.unsqueeze(
            1
        )  # Add a dimension for the transformer (batch_size, seq_len, embed_dim)
        x1 = self.transformer1(
            x
        )  # Transformer expects input of shape (seq_len, batch_size, embed_dim)
        x2 = self.transformer2(x1)
        print(x2)
        embedding = x2[:, -1, :]  # Get the last output token
        x2 = self.fc(embedding)  # Fully connected layer on the last output token
        return x2.squeeze(-1), embedding  # Return the final output and the embeddings


# Prepare Data
dataset = FinancialDataset(first_month_data[:100])
dataloader = DataLoader(dataset, batch_size=64, shuffle=False)

# Initialize Model
model = TransformerModel(input_dim=8, embed_dim=64, n_heads=8, ff_dim=256, n_layers=4)


# Training Loop
optimizer = torch.optim.RAdam(model.parameters(), lr=0.001)
criterion = nn.L1Loss()

for epoch in range(1):
    for batch in dataloader:
        optimizer.zero_grad()
        output, _ = model(batch["features"])
        loss = criterion(output, batch["target"])
        # print(f'loss: {loss}')
        loss.backward()
        optimizer.step()

print("Training Complete")

tensor([[[ 0.0992,  1.4843, -0.1232,  ...,  1.0908,  0.5693, -0.7092]],

        [[-0.0376,  1.4571, -0.4478,  ...,  1.2565,  0.1120, -0.3278]],

        [[ 0.1024,  1.2096, -0.4553,  ...,  0.7368,  0.0795, -0.3238]],

        ...,

        [[-0.0818,  1.4794, -0.1962,  ...,  1.3378,  0.3353, -0.1954]],

        [[-0.0091,  0.8141, -0.1943,  ...,  0.8506,  0.1997, -0.3229]],

        [[-0.1247,  1.8256, -0.1752,  ...,  1.0059,  0.1644, -0.6466]]],
       grad_fn=<NativeLayerNormBackward0>)
tensor([[[-0.4026,  1.9329, -0.7334, -0.3452,  0.1807,  0.6965,  1.0397,
          -1.6557, -0.2872, -0.7365, -0.3029, -0.6215,  0.1576, -0.7920,
          -0.9099,  0.1068, -0.2182, -0.9330, -0.3531, -1.6048, -1.4847,
          -1.1427, -0.1009, -0.3589,  1.5950, -2.5913,  0.1493, -0.8947,
          -0.9505,  0.1281,  1.6563,  2.5651, -0.8410, -0.1515,  0.2405,
          -1.5606,  1.0797, -0.7109, -1.7710,  0.2491,  0.3771, -0.2357,
           0.0651,  0.3499,  0.6788,  1.2481,  2.7510,  0.9215, -0.

/tmp/ipykernel_155/2190432382.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.data['log_return'] = pd.to_numeric(self.data['log_return'], errors='coerce')
